# OpenAI Function Calling In LangChain 🤖

## Introduction
This notebook demonstrates how to `use the LangChain framework to integrate OpenAI function calling capabilities`. We'll cover the setup, the syntax of Pydantic data classes, and how to convert these data classes into OpenAI functions. Additionally, we'll explore how to bind and invoke these functions within LangChain models.

### Setup the Environment, OpenAI API Key  and Imports
First, we need to import the necessary libraries and set up the Environemnt and the OpenAI API key:

In [1]:
# Import necessary libraries
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a .env file
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# Print OpenAI API key (masked)
print(f"OPENAI_API_KEY: {os.getenv('OPENAI_API_KEY')[:5]}*****")

OPENAI_API_KEY: sk-ft*****


**Note** Ensure you have the required packages installed:
```py
%pip install pydantic==1.10.8
%pip install rich
```

In [2]:
# Import necessary modules from Pydantic 
from typing import List
from pydantic import BaseModel, Field

In [3]:
# Import necessary modules from rich library that helps to improve the readability of nested dictionary outputs
from rich import print
from rich.pretty import Pretty

### Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

- Example: Creation of a class  
With Python, you can define the class as follows:

In [4]:
# In Python you can create a claas like that: 
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [6]:
# Example 1  usage
foo = User(name="Pinco",age=32, email="pinc@pall.com")
#print(foo.name)
foo.name



'Pinco'

In [7]:
# Example 2 usage
foo = User(name="Pinco",age="bar", email="pinc@pall.com")
foo.age

'bar'

**Note**: The above line does raise an error because 'age' should be an integer.

With Pydantic, you can define the class as follows:

In [8]:
# Pydantic version of the User class
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [9]:
# Example usage
foo_p = pUser(name="Jane", age=32, email="jane@pall.com")
foo_p.name

'Jane'

In [10]:
# Define a class that contains a list of pUser instances
class Class(BaseModel):
    students: List[pUser]

In [11]:
# Create an instance of the Class with one student
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@pall.com")]
)

In [12]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@pall.com')])

## Pydantic to OpenAI function definition


We can convert Pydantic data classes into OpenAI function definitions.

In [13]:
# Define a Pydantic data class for a weather search function
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [19]:
# Import the function to convert Pydantic data classes to OpenAI function definitions
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [15]:
# Convert the Pydantic data class to an OpenAI function
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

C:\Users\Michela\AppData\Local\Temp\ipykernel_8892\1313296718.py:1: LangChainDeprecationWarning: The function `_convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  weather_function = convert_pydantic_to_openai_function(WeatherSearch)


In [20]:
# Display the converted function definition
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [21]:
# Display the function definition in a pretty format
print(Pretty(weather_function))


{
    'name': 'WeatherSearch',
    'description': 'Call this with an airport code to get the weather at that airport',
    'parameters': {
        'properties': {'airport_code': {'description': 'airport code to get weather for', 'type': 'string'}},
        'required': ['airport_code'],
        'type': 'object'
    }
}

In [22]:
# Define another Pydantic data class for a weather search function with a simpler description
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

In [23]:
# Another example of a Pydantic data class for a weather search function
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [24]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [25]:
response = convert_pydantic_to_openai_function(WeatherSearch2)
print(Pretty(response))

{
    'name': 'WeatherSearch2',
    'description': 'Call this with an airport code to get the weather at that airport',
    'parameters': {
        'properties': {'airport_code': {'type': 'string'}},
        'required': ['airport_code'],
        'type': 'object'
    }
}

In [26]:
from langchain.chat_models import ChatOpenAI

In [27]:
model = ChatOpenAI()

C:\Users\Michela\AppData\Local\Temp\ipykernel_8892\1603725051.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI()


**Note** weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [29]:
model.invoke("what is the weather in Amsterdam today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"EHAM"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 70, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-56896ff6-34cc-4923-8de3-07a6d554bf62-0')

In [30]:
response = model.invoke("what is the weather in Amsterdam  today?", functions=[weather_function])
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"AMS"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 17,
            'prompt_tokens': 71,
            'total_tokens': 88,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-f2ca520e-6fee-4671-ade0-643273e1bda8-0'
)

In [31]:
model_with_function = model.bind(functions=[weather_function])

In [32]:
model_with_function.invoke("what is the weather in Amsterdam?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"AMS"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 69, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-d24a1c8d-0778-4721-9010-e759ef87e74c-0')

In [33]:
response = model_with_function.invoke("what is the weather in Amsterdam?")
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"EHAM"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 18,
            'prompt_tokens': 69,
            'total_tokens': 87,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-68468615-6a6e-4b33-91ee-29025ff9e68c-0'
)

## Forcing the Model to Use a Function

We can force the model to use a specific function when invoking it.

**Note** model_with_function = model.bind(functions=[weather_function])

In [34]:
# Bind the model to use a specific function when invoking it
# This ensures that the model always uses the specified function
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [35]:
# Invoke the model with a query, forcing it to use the WeatherSearch function
model_with_forced_function.invoke("what is the weather in Amsterdam?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"EHAM"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 79, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-99cfd657-5cb9-404c-9800-ed0441bc524c-0')

In [36]:
# Display the response in a pretty format using the rich library
response = model_with_forced_function.invoke("what is the weather in Amsterdam?")
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"EHAM"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 8,
            'prompt_tokens': 79,
            'total_tokens': 87,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-0f29949a-64e0-48f1-9267-9be048ba27e8-0'
)

**Explanation output**  This output is a structured representation of an AIMessage object.This output indicates that the AI model invoked a function named WeatherSearch with an argument of {"airport_code":"EHAM"} and provides detailed metadata about the token usage and model.  
Interesting is to have a look at the structure of the AIMessage:   
Class: AIMessage  
Attributes:  
  - content: An empty string, indicating no direct textual content in the message.    
  - additional_kwargs: A dictionary containing additional keyword arguments:  
      - function_call: A dictionary specifying:
          - arguments: A JSON string with the argument {"airport_code":"SFO"}, indicating the airport code for San Francisco.
      - name: The function name WeatherSearch.    

response_metadata: A dictionary with metadata about the response:  
  - token_usage: A dictionary detailing token usage:
completion_tokens: 8
prompt_tokens: 79
total_tokens: 87
completion_tokens_details: A dictionary with various token counts, all set to 0.
prompt_tokens_details: A dictionary with audio_tokens and cached_tokens, both set to 0.
  - model_name: The model used is gpt-3.5-turbo.
  - system_fingerprint: None.
  - finish_reason: The reason for finishing is stop.
  - logprobs: None.  
  
id: A unique identifier for the run: run-cc94ce3d-0e5c-4f99-9a20-5cdae01e5231-0.

In [37]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 74, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d8679ffb-8f91-41e1-9543-e1eb66acd395-0')

In [38]:
# Another invocation example with a different query
# The model is still forced to use the WeatherSearch function
response = model_with_forced_function.invoke("hi!")
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"JFK"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 8,
            'prompt_tokens': 74,
            'total_tokens': 82,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-e246a53c-bd68-4ac0-969f-7f3409c4cbf3-0'
)

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [39]:
# Import the necessary module for creating chat prompts
from langchain.prompts import ChatPromptTemplate

In [40]:
# Create a chat prompt template with predefined messages
# The system message sets the behavior of the assistant
# The user message is a placeholder for the user's input
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [41]:
# Create a chain that uses the model bound to the weather function
# The chain will use the prompt template to generate responses
chain = prompt | model_with_function

In [42]:
# Invoke the chain with an example input
chain.invoke({"input": "what is the weather in Amsterdam?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"EHAM"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 75, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-15f78462-fdfc-439b-b5a6-e4b227375650-0')

In [43]:
# Print the response in a pretty format using the rich library
response = chain.invoke({"input": "what is the weather in Amsterdam?"})
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"EHAM"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 18,
            'prompt_tokens': 75,
            'total_tokens': 93,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-d4f8cb44-9878-4f2c-8cb4-04ae377806a5-0'
)

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [44]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

Let's convert Pydantic data classes to OpenAI function definitions and stores them in a list named `functions`. Each function can be used for specific tasks such as weather search and artist search in the OpenAI model.

In [45]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch), # Converts WeatherSearch Pydantic class to OpenAI function
    convert_pydantic_to_openai_function(ArtistSearch),  # Converts ArtistSearch Pydantic class to OpenAI function
]

Let's binds a list of predefined functions to an OpenAI model. The 'functions' list contains converted Pydantic data classes that define specific tasks (e.g., WeatherSearch, ArtistSearch). Binding these functions to the model enables the model to invoke them during its execution for specific queries.

In [46]:
model_with_functions = model.bind(functions=functions)

Next, let's  snippet invoke the `model_with_functions` with the query "what is the weather in sf?". The model is expected to use the bound functions (e.g., WeatherSearch) to process this query. The query will trigger the model to call the appropriate function to get the weather information for San Francisco (SFO).

In [47]:
model_with_functions.invoke("what is the weather in Amsterdam?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"AMS"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 116, 'total_tokens': 133, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-9ceef3a6-5fd6-44c8-a784-b67d40ca5f9d-0')

In [48]:
response = model_with_functions.invoke("what is the weather in Amsterdam?")
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"airport_code":"AMS"}', 'name': 'WeatherSearch'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 17,
            'prompt_tokens': 116,
            'total_tokens': 133,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-5d59a48c-7884-466b-a321-1a7639088e88-0'
)

In [49]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Taylor Swift","n":3}', 'name': 'ArtistSearch'}}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 118, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-5e67ca2f-2879-49be-8eb3-41eb1191d019-0')

In [50]:
response = model_with_functions.invoke("what are three songs by taylor swift?")
print(Pretty(response))

AIMessage(
    content='',
    additional_kwargs={
        'function_call': {'arguments': '{"artist_name":"Taylor Swift","n":3}', 'name': 'ArtistSearch'}
    },
    response_metadata={
        'token_usage': {
            'completion_tokens': 22,
            'prompt_tokens': 118,
            'total_tokens': 140,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-184236a4-c34e-4aa8-84e8-da6f78eb2aeb-0'
)

In [51]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 111, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ea025e56-6949-4265-96d6-e79ffbbc75ac-0')

In [52]:
response = model_with_functions.invoke("hi!")
print(Pretty(response))

AIMessage(
    content='Hello! How can I assist you today?',
    additional_kwargs={},
    response_metadata={
        'token_usage': {
            'completion_tokens': 11,
            'prompt_tokens': 111,
            'total_tokens': 122,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run-fd4053ed-f6a0-438f-9001-392a1a37982b-0'
)

## Conclusion
In this notebook, we demonstrated how to `use LangChain to integrate OpenAI function calling capabilities`. We covered the setup, Pydantic data classes, converting these classes into OpenAI functions, and invoking these functions within LangChain models. These examples showcase the flexibility and power of LangChain in building advanced language models and workflows.